In [10]:
import pandas as pd
import numpy as np

### Les données LPS

In [17]:
fusion_3 = "../../DATABASE/basket/LPS/3_fusion.csv"
fusion_4 = "../../DATABASE/basket/LPS/5_fusion.csv"
lps_3 = pd.read_csv(fusion_4, sep=";")
lps_3

,timestamp,lat_brute,long_brute,latitude_fusion,longitude_fusion,hdop,vitesse_fusion,battery
0,1702478113100,NaN,NaN,NaN,NaN,-1.0,-1.0,0
1,1702478113200,NaN,NaN,NaN,NaN,-1.0,-1.0,0
2,1702478113300,NaN,NaN,NaN,NaN,-1.0,-1.0,0
3,1702478113400,NaN,NaN,NaN,NaN,-1.0,-1.0,0
4,1702478113500,NaN,NaN,NaN,NaN,-1.0,-1.0,0
...,...,...,...,...,...,...,...,...
87472,1702486860300,NaN,NaN,NaN,NaN,-1.0,-1.0,0
87473,1702486860400,NaN,NaN,NaN,NaN,-1.0,-1.0,0
87474,1702486860500,NaN,NaN,NaN,NaN,-1.0,-1.0,0
87475,1702486860600,NaN,NaN,NaN,NaN,-1.0,-1.0,0


In [12]:
lps_3['lat_brute'].unique()

array([nan])

### Les données tracking

In [9]:
tracking_1_1_2_1 = "../../DATABASE/basket/TRACKING/Equipe1_Vague1_Poss2_video_1.txt"
tracking_df = pd.read_csv(tracking_1_1_2_1, sep=",")
tracking_df

,0,0.1,10.7385311127,2.4233112335
0,0,1,10.758081,7.556952
1,0,2,9.685260,7.294818
2,0,3,6.877824,9.319980
3,0,4,5.604179,9.292738
4,0,5,6.155215,9.916403
...,...,...,...,...
3281,385,3,5.606390,1.841755
3282,385,4,0.658973,5.352539
3283,385,6,2.336708,9.233820
3284,385,7,2.905753,8.886047


In [5]:
tracking_df["timestamp"] = tracking_df["frame"] / 25

tracking_df = tracking_df.dropna(subset=["latitude_fusion", "longitude_fusion"])

tracking_df["datetime"] = pd.to_datetime(tracking_df["timestamp"], unit="ms")

tracking_df = tracking_df.sort_values("datetime")
time_diffs = tracking_df["datetime"].diff().dropna().dt.total_seconds()

KeyError: ['latitude_fusion', 'longitude_fusion']

J'ai un problème et je voudrais qu'on le resolve ensemble. J'ai des données de tracking qui se presentent comme suite: 0,0,10.7385311127,2.4233112335
0,1,10.7580814362,7.5569515228
0,2,9.6852598190,7.2948179245
0,3,6.8778243065,9.3199796677
0,4,5.6041789055,9.2927379608
0,5,6.1552147865,9.9164028168
0,6,5.9527912140,5.0271210670
0,7,4.8486547470,5.5767488480
0,8,7.7850975990,10.6194810867
1,0,10.7934675217,2.4391222000
1,1,10.7580795288,7.5570054054
1,2,9.7006597519,7.3018217087
1,3,6.9694957733,9.3055181503
1,4,5.6162209511,9.2919712067
1,5,6.1434712410,9.9335775375
1,6,5.9752697945,5.0216503143
1,7,4.8483066559,5.5850920677
1,8,7.7175521851,10.6883754730
2,0,10.8149042130,2.4565124512
2,1,10.7579250336,7.5626974106
2,2,9.7036275864,7.2909293175
2,3,6.9925117493,9.3433561325
2,4,5.5050201416,9.2996940613
2,5,6.1421518326,9.9293451309
2,6,5.9690389633,5.0267295837
2,7,4.9030656815,5.5913071632
2,8,7.7793993950,10.7771663666
3,0,10.8322410583,2.4760200977
3,1,10.7580280304,7.5589027405
3,2,9.7042589188,7.2874441147
3,3,7.0315570831,9.4002285004
3,4,5.5206923485,9.2985353470
3,5,6.1331715584,9.8932018280
3,6,5.9660601616,5.0353903770
3,7,4.9405260086,5.6021599770
3,8,7.8388147354,10.8428821564
4,0,10.8481369019,2.4955701828
4,1,10.7577753067,7.5682272911
4,2,9.7041177750,7.2921996117
4,3,7.0407762527,9.4762802124
4,4,5.5661320686,9.3024320602
4,5,6.0577692986,9.8546276093
4,6,5.9655728340,5.0381984711
4,7,4.9715533257,5.6070280075
4,8,7.8867015839,10.9132404327
5,0,10.8521080017,2.5219168663
5,1,10.7576885223,7.5714259148
5,2,9.7038002014,7.2939696312
5,3,7.0414981842,9.5321235657
5,4,5.5833272934,9.2902088165
5,5,5.9673075676,9.8820676804
5,6,5.9561491013,5.0378751755
5,7,4.9805154800,5.6085066795
5,8,7.9464406967,10.9513139725
6,0,10.8413696289,2.5373182297
6,1,10.7686166763,7.5775523186
6,2,9.7283544540,7.3115062714
6,3,7.0540547371,9.6290063858
6,4,5.6262450218,9.2635583878
6,5,5.6707224846,9.9381923676
6,6,5.9611568451,5.0449938774
6,7,5.0020213127,5.6036634445
6,8,7.9713869095,10.9919853210
7,0,10.8364753723,2.5618174076
7,1,10.7722949982,7.5897612572
7,2,9.7368679047,7.3176136017
7,3,7.0470185280,9.7297248840
7,4,5.6245012283,9.2489137650
7,5,5.6346940994,9.9320812225
7,6,5.9633264542,5.0474233627
7,7,4.9704017639,5.6050782204
7,8,7.9642429352,10.9810428619
8,0,10.8445920944,2.5826449394
8,1,10.7737331390,7.5889010429
8,2,9.7395944595,7.3144593239
8,3,7.0413484573,9.8303880692
8,4,5.6146163940,9.2443571091
8,5,5.6341681480,9.9356164932
8,6,5.9640526772,5.0481185913
8,7,4.9670672417,5.6062874794
8,8,7.9321537018,11.1609792709
9,0,10.8469247818,2.6088972092
9,1,10.7742166519,7.5884690285
9,2,9.7515850067,7.3449292183
9,3,7.0348677635,9.9725503922
9,4,5.6021103859,9.2371835709
9,5,5.8336095810,9.9304981232
9,6,5.9644231796,5.0482969284
9,7,4.9651460648,5.6130313873
9,8,7.9239850044,11.2914543152
10,0,10.8465986252,2.6467039585
10,1,10.7639122009,7.5683150291
10,2,9.7671918869,7.3769044876
10,3,7.0245103836,10.0229377747
10,4,5.5974111557,9.2285051346
10,5,6.0337405205,9.8976011276
10,6,5.9645657539,5.0543155670
10,7,4.9638142586,5.6154828072
10,8,7.9388279915,11.2144670486 dans les fichiers equipe1.....txt et chaque fichier a en appui deux videos prisent dans des angles differents. Ensuite nous avons des fichiers fusion.csv appartenant chacun a un joueur qui contient des infos qui se trouveront dans les données de tracking et l'objectif est de trouver un moyen faire correspondre le bon fichier tracking au bon fichier fusion.csv de sorte a pouvoir identifier les moments et positions exactes sur les deux fichiers. voici un peu le contenu d'un fichier fusion: timestamp;lat_brute;long_brute;latitude_fusion;longitude_fusion;hdop;vitesse_fusion;battery
1702478113100;;;;;-1.0;-1.0;0
1702478113200;;;;;-1.0;-1.0;0
1702478113300;;;;;-1.0;-1.0;0
1702478113400;;;;;-1.0;-1.0;0
1702478113500;;;;;-1.0;-1.0;0
1702478113600;;;;;-1.0;-1.0;0
1702478113700;;;;;-1.0;-1.0;0
1702478113800;;;;;-1.0;-1.0;0
1702478113900;;;;;-1.0;-1.0;0
1702478114000;;;;;-1.0;-1.0;0
1702478114100;;;;;-1.0;-1.0;0
1702478114200;;;;;-1.0;-1.0;0
1702478114300;;;;;-1.0;-1.0;0
1702478114400;;;;;-1.0;-1.0;0
1702478114500;;;;;-1.0;-1.0;0
1702478114600;;;;;-1.0;-1.0;0
1702478114700;;;;;-1.0;-1.0;0
1702478114800;;;;;-1.0;-1.0;0
1702478114900;;;;;-1.0;-1.0;0
1702478115000;;;;;-1.0;-1.0;0
1702478115100;;;;;-1.0;-1.0;0
1702478115200;;;;;-1.0;-1.0;0
1702478115300;;;;;-1.0;-1.0;0
1702478115400;;;;;-1.0;-1.0;0
1702478115500;;;;;-1.0;-1.0;0
1702478115600;;;;;-1.0;-1.0;0
1702478115700;;;;;-1.0;-1.0;0
1702478115800;;;;;-1.0;-1.0;0
1702478115900;;;;;-1.0;-1.0;0
1702478116000;;;;;-1.0;-1.0;0
1702478116100;;;;;-1.0;-1.0;0
1702478116200;;;;;-1.0;-1.0;0
1702478116300;;;;;-1.0;-1.0;0
1702478116400;;;;;-1.0;-1.0;0
1702478116500;;;;;-1.0;-1.0;0
1702478116600;;;;;-1.0;-1.0;0
1702478116700;;;;;-1.0;-1.0;0
1702478116800;;;;;-1.0;-1.0;0
1702478116900;;;;;-1.0;-1.0;0
1702478117000;;;;;-1.0;-1.0;0
1702478117100;;;;;-1.0;-1.0;0
1702478117200;;;;;-1.0;-1.0;0
1702478117300;;;;;-1.0;-1.0;0
1702478117400;;;;;-1.0;-1.0;0
1702478117500;;;;;-1.0;-1.0;0
1702478117600;;;;;-1.0;-1.0;0
1702478117700;;;;;-1.0;-1.0;0
1702478117800;;;;;-1.0;-1.0;0
1702478117900;;;;;-1.0;-1.0;0

In [ ]:
tracking_df

,timestamp,lat_brute,long_brute,latitude_fusion,longitude_fusion,hdop,vitesse_fusion,battery,datetime
266,1702478139700,NaN,NaN,49.458047,1.062343,0.21,-1.000,5,2023-12-13 14:35:39.700
267,1702478139800,NaN,NaN,49.458045,1.062340,0.19,-1.000,4,2023-12-13 14:35:39.800
268,1702478139900,NaN,NaN,49.458045,1.062340,0.18,-1.000,4,2023-12-13 14:35:39.900
269,1702478140000,NaN,NaN,49.458045,1.062341,0.17,-1.000,4,2023-12-13 14:35:40.000
270,1702478140100,NaN,NaN,49.458045,1.062342,0.16,-1.000,4,2023-12-13 14:35:40.100
...,...,...,...,...,...,...,...,...,...
87472,1702486860300,NaN,NaN,49.457994,1.062402,0.26,1.644,5,2023-12-13 17:01:00.300
87473,1702486860400,NaN,NaN,49.457993,1.062404,0.28,1.673,5,2023-12-13 17:01:00.400
87474,1702486860500,NaN,NaN,49.457993,1.062406,0.30,1.698,5,2023-12-13 17:01:00.500
87475,1702486860600,NaN,NaN,49.457992,1.062409,0.32,1.709,5,2023-12-13 17:01:00.600


In [ ]:
fusion_3_track = tracking_df[tracking_df["id"] == 3]

In [ ]:
fusion_3_track

,frame,id,x,y
3,0,3,6.877824,9.319980
12,1,3,6.969496,9.305518
21,2,3,6.992512,9.343356
30,3,3,7.031557,9.400229
39,4,3,7.040776,9.476280
...,...,...,...,...
3253,381,3,5.730844,1.894935
3261,382,3,5.698042,1.883440
3268,383,3,5.661434,1.857994
3275,384,3,5.639479,1.852672
